<a href="https://colab.research.google.com/github/ArunDharavath/ML-DL/blob/main/NLP/Custom_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy==3.6.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 14.6 MB/s eta 0:00:00


In [2]:
!python -m spacy info

2024-01-14 07:01:56.790180: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 07:01:56.790881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 07:01:56.794717: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 07:01:56.811024: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-14 07:01:59.837448: W tensorflow/compiler/tf2

In [3]:
#load custom annotated data

import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

text = json.load(open('/content/drive/MyDrive/ColabNotebooks/Custom_NER/annotations/annotations.json','r'))

# Display the first item in the dataset
text[0]

['Skip to main content\nSBNation.com homepage\nLog in or sign up\nSite searchSearch\nSBNation.com main menu\nBLOGS\nSECRET BASE\nDRAFTKINGS \nNFL \nNBA \nCBB \nCFB \nGOLF\nWNBA \nMLB \nSOCCER \nUFC \nF1\nMORE \nSearch\nFILED UNDER:\nNFL\n \nNFL PLAYOFFS\n \nDRAFTKINGS\n14 NFL playoff teams, ranked by their Super Bowl chances\nThe SB Nation staff got together to rank all 14 teams by their chances of winning Super Bowl 58',
 {'entities': []}]

In [4]:
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

  return db

#Split the data into train and test for better generalization
from sklearn.model_selection import train_test_split
train, test = train_test_split(text, test_size=0.2)

# Display the number of items in the training and testing sets
len(train), len(test)

# Open a file to log errors during annotation processing
file = open('/content/drive/MyDrive/ColabNotebooks/Custom_NER/trained_models/train_file.txt','w')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('/content/drive/MyDrive/ColabNotebooks/Custom_NER/trained_models/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('/content/drive/MyDrive/ColabNotebooks/Custom_NER/trained_models/test_data.spacy')

# Close the error log file
file.close()

100%|██████████| 15/15 [00:00<00:00, 357.06it/s]


In [5]:
# Creating a config file for NER task

!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

2024-01-14 07:02:28.463541: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 07:02:28.463626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 07:02:28.465135: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 07:02:29.962847: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- 

In [6]:
#Running the model on the custom data

!python -m spacy train config.cfg  --output /content/drive/MyDrive/ColabNotebooks/Custom_NER/trained_models/output  --paths.train /content/drive/MyDrive/ColabNotebooks/Custom_NER/trained_models/train_data.spacy  --paths.dev /content/drive/MyDrive/ColabNotebooks/Custom_NER/trained_models/test_data.spacy

2024-01-14 07:02:38.979630: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 07:02:38.979731: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 07:02:38.981695: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 07:02:41.052576: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory:
/content/drive/MyDrive/ColabNotebooks/Custom_NER/trained_models/output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2024-01-14 07:02:44,582] [INFO] Set up nlp objec

In [7]:
#Load the model with the best parameters

nlp_ner = spacy.load('/content/drive/MyDrive/ColabNotebooks/Custom_NER/trained_models/output/model-best')

In [8]:
#Testing data fed into the model

doc = nlp_ner('''Can't-Miss Play: 59-yard pass! Love dots Reed in stride for field-stretching strike
Former NFL player and scout Bucky Brooks knows the ins and outs of this league, providing keen insight in his notebook. In today's installment, he spotlights 14 players who could tilt the balance in the hunt for the Lombardi Trophy ...

The NFL playoffs are nearly upon us! Super Wild Card Weekend provides an enticing six-pack of games over three days. But before we dive into this single-elimination tournament, I'd like to focus attention on a series of players whose postseason performance could prove more consequential than you might realize.

Obviously, football's biggest stage is built for football's biggest stars, but the postseason also has a way of providing opportunities for lesser-heralded players to make significant contributions. The big names always dominate discussion, but underrated playmakers consistently assume outsized roles in deciding which team ultimately hoists the Lombardi Trophy on Super Bowl Sunday.

With that in mind, I studied each of the 14 rosters in this tournament field to identify one X-factor on each team. Don't be surprised if some of these guys steal the spotlight!

RELATED LINKS
Next Gen Stats 2023 All-Pro Team: Browns, Chiefs, Williams bros head up defense; plus, special teamers
AFC X-factors
Seed
1
Baltimore Ravens
Baltimore Ravens
13-4
Odell Beckham
Odell Beckham
WR · Year 10
Though Beckham is no longer regarded as a superstar, the three-time Pro Bowler is a "bright lights, big stage" performer. The veteran led the Ravens with an average of 16.1 yards per catch this season, proving he remains a serious threat to deliver a big play in the passing game on a crafty double move or catch-and-run concept. With many opponents still uncertain how to best defend Baltimore's multi-faceted attack, OBJ could turn back the clock during the postseason with a sensational run that reminds the football world of his insane playmaking ability in his prime.

Please enable Javascript to view this content
Seed
2
Buffalo Bills
Buffalo Bills
11-6
James Cook
James Cook
RB · Year 2
The Bills cannot solely rely on Josh Allen's heroics to carry them through the playoffs. Cook can lighten the quarterback's load by adding some pop to the offense as a dynamic runner/receiver with big-play potential. The first-time Pro Bowler amassed 1,567 scrimmage yards and six total scores as Buffalo's top back, and the Bills need the young playmaker to balance an offense that too often banks on No. 17 to spin individual magic in key moments.

Seed
3
Kansas City Chiefs
Kansas City Chiefs
11-6
George Karlaftis
George Karlaftis
DE · Year 2
The second-year pro has emerged as a capable sidekick to all-world DT Chris Jones, providing Kansas City with a disruptive playmaker off the edge. With 10.5 sacks, 17 QB hits and seven tackles for loss in 16 games, Karlaftis is a destructive force at the point of attack with splash-play potential. As the Chiefs engage in more grind-it-out affairs due to offensive struggles, this energetic edge could tilt the game in K.C.'s favor with a strip-sack in crunch time.

Seed
4
Houston Texans
Houston Texans
10-7
Devin Singletary
Devin Singletary
RB · Year 5
The extra attention on rookie sensation C.J. Stroud could force the young Texans to rely on a more balanced attack in order to advance in the playoffs. Singletary's production as a shifty RB1 could create more one-on-one opportunities for Nico Collins and others in the passing game when opponents drop an extra defender in the box. Given Houston's unbeaten record (3-0) when Singletary surpasses the 100-yard mark, the 26-year-old back could get a heavy workload in the postseason.

Seed
5
Cleveland Browns
Cleveland Browns
11-6
David Njoku
David Njoku
TE · Year 7
The insertion of Joe Flacco into the lineup transformed Njoku into a prolific playmaker between the hashes. The former first-round pick, who just made his first Pro Bowl in Year 7, has 28 catches for 373 yards and four scores over his past four games, with Cleveland's new veteran gunslinger prioritizing the big-bodied tight end in the passing game. As opponents attempt to contain the Browns' punishing ground attack while tilting coverage in Amari Cooper's direction, Njoku could emerge as the go-to guy this offense leans on during a playoff run.

Seed
6
Miami Dolphins
Miami Dolphins
11-6
Christian Wilkins
Christian Wilkins
DT · Year 5
The injury losses of edge-rushing studs Bradley Chubb and Jaelan Phillips put the onus on Wilkins to wreak havoc on opponents as an interior pass rusher. The former Clemson star possesses freakish athleticism and a destructive game, but the Dolphins will need more pure dominance from Wilkins in order to slow down the explosive offenses in the AFC playoff field. If he can up the ante as the designated playmaker up front, Miami can move through the tournament with Vic Fangio masterfully managing an injury-riddled defense.

Seed
7
Pittsburgh Steelers
Pittsburgh Steelers
10-7
Alex Highsmith
Alex Highsmith
OLB · Year 4
With T.J. Watt ruled out this week, Pittsburgh needs Highsmith to transition from Robin to Batman. The former third-round pick has posted 27.5 sacks over the past three seasons as a complementary pass rusher, but now he must flourish in the lead role. As Mike Tomlin concocts a plan to mitigate the loss of a three-time NFL sacks leader -- Watt racked up a league-high 19 QB takedowns this season -- Highsmith will play a pivotal role in the team's defensive tactics.

NFC X-factors
Seed
1
San Francisco 49ers
San Francisco 49ers
12-5
Deommodore Lenoir
Deommodore Lenoir
CB · Year 3
The high-powered offenses in the NFC will test the talent and depth of the 49ers' defensive backfield. As the CB2 on a defense that excels in coverage, Lenoir will see plenty of action as quarterbacks opt to throw away from Charvarius Ward. In his third NFL season, the former fifth-round pick has recorded 10 passes defensed and three interceptions. Now the 49ers will need the 5-foot-10, 200-pounder to be prepared for a host of WR2s with big-play potential on the perimeter -- guys like Puka Nacua, DeVonta Smith, Chris Godwin and Brandin Cooks, among others.

Seed
2
Dallas Cowboys
Dallas Cowboys
12-5
Tyron Smith
Tyron Smith
LT · Year 13
The eight-time Pro Bowler has been in and out of the lineup in recent seasons, but the Cowboys need Smith's presence at left tackle to fortify an offensive line tasked with protecting Dak Prescott, an All-Pro-caliber passer who has sizzled down the stretch. As a rugged blocker with heavy hands and knockout power, the 33-year-old still has the ability to handle the heavyweight edge rushers on the NFC side of the playoff bracket.

Seed
3
Detroit Lions
Detroit Lions
12-5
Jahmyr Gibbs
Jahmyr Gibbs
RB · Rookie
With David Montgomery and Gibbs, the Lions boast a spectacular 1-2 backfield punch that is problematic for opponents attempting to neutralize the Jared Goff-led aerial attack. The soft coverage needed to slow down Amon-Ra St. Brown and Co. leaves plenty of space for the two RBs to chew up yardage as runner/receivers on the perimeter. The reason I'm spotlighting Gibbs, in particular, as Detroit's X-factor? With a robust average of 5.2 yards per carry, 52 catches and 11 total touchdowns, this rookie is a true home run hitter who can change the tenor of a game with one touch. I can't wait to see what creative offensive coordinator Ben Johnson does with such a potent weapon in his first postseason.

Seed
4
Tampa Bay Buccaneers
Tampa Bay Buccaneers
9-8
Antoine Winfield Jr.
Antoine Winfield Jr.
S · Year 4
The do-it-all playmaker fills the stat sheet as a safety with elite blitzing, tackling and coverage ability. Winfield can impact the game as an extra pass rusher (SEE: six sacks this season) or deep-middle defender (three interceptions and 12 passes defensed), while also flashing impressive skills as a rock-solid tackler (tied for the league lead with six forced fumbles). Considering how Todd Bowles wants to bring pressure from every angle, Winfield's versatility could help the Buccaneers make a surprising run this January.

Seed
5
Philadelphia Eagles
Philadelphia Eagles
11-6
D'Andre Swift
D'Andre Swift
RB · Year 4
After the Eagles' late-season swoon, Nick Sirianni could decide to hand the ball to Swift early and often in an attempt to re-establish the run-heavy approach that propelled the team to a fast start in 2023. The electric back flashes big-play potential with the ball in his hands on inside and outside runs, but the Eagles must commit to giving him 20-plus touches in order to maximize his potential. Given the pressure on the reigning NFC champions to avoid a one-and-done appearance, Sirianni could increasingly rely on Swift and the ground game to reverse the team's fortunes in the playoffs.

Seed
6
Los Angeles Rams
Los Angeles Rams
10-7
Byron Young
Byron Young
OLB · Rookie
The attention on Aaron Donald and fellow rookie Kobie Turner as interior disruptors will create plenty of opportunities for Young off the edge. The third-round pick has eight sacks, eight tackles for loss and 19 quarterback hits in 17 games, displaying exceptional first-step quickness and closing burst. If the Rams force opponents into long-yardage situations, Young could change the game with a splash play in a critical moment.

Seed
7
Green Bay Packers
Green Bay Packers
9-8
Jayden Reed
Jayden Reed
WR · Rookie
Green Bay's leading receiver has emerged as a reliable weapon on the perimeter in Year 1. Reed's sticky hands and superb route-running skills have endeared him to Jordan Love as the Packers' temporary WR1. (Christian Watson is the top dog when healthy.) With this young team bringing a "nothing to lose" attitude into the tournament, Reed has a chance to be the difference maker who helps the Packers re-emerge as true contenders.

Watch Live Games on NFL+
Stream this week's games on your phone or tablet.
Your Available Games


Browns at Texans
Saturday, January 13th • 2:30 PM


Dolphins at Chiefs
Saturday, January 13th • 6:10 PM


Steelers at Bills
Sunday, January 14th • 11:00 AM


Packers at Cowboys
Sunday, January 14th • 2:30 PM


Rams at Lions
Sunday, January 14th • 6:15 PM


Eagles at Buccaneers
Monday, January 15th • 6:15 PM

TEAMS
AFC NORTH
Baltimore Ravens
Cincinnati Bengals
Cleveland Browns
Pittsburgh Steelers
AFC EAST
Buffalo Bills
Miami Dolphins
New England Patriots
New York Jets
AFC SOUTH
Houston Texans
Indianapolis Colts
Jacksonville Jaguars
Tennessee Titans
AFC WEST
Denver Broncos
Kansas City Chiefs
Las Vegas Raiders
Los Angeles Chargers
NFC NORTH
Chicago Bears
Detroit Lions
Green Bay Packers
Minnesota Vikings
NFC EAST
Dallas Cowboys
New York Giants
Philadelphia Eagles
Washington Commanders
NFC SOUTH
Atlanta Falcons
Carolina Panthers
New Orleans Saints
Tampa Bay Buccaneers
NFC WEST
Arizona Cardinals
Los Angeles Rams
San Francisco 49ers
Seattle Seahawks ''')

In [9]:
#NER prediction on test data

spacy.displacy.render(doc, style='ent', jupyter=True)